Python 3 明确区分了人类可读的文本字符串和原始的字节序列。隐式地把字节序列转换成 Unicode 文本已成过去。

## 4.1 字符问题

“字符串”是个相当简单的概念：一个字符串是一个字符序列。

在 2015 年，“字符”的最佳定义是 Unicode 字符。因此，从 Python 3 的str 对象中获取的元素是 Unicode 字符，这相当于从 Python 2 的unicode 对象中获取的元素，而不是从 Python 2 的 str 对象中获取的原始字节序列。

Unicode 标准把字符的标识和具体的字节表述进行了如下的明确区分：

* 字符的标识，即码位，是 0~1 114 111 的数字（十进制），在Unicode 标准中以 4~6 个十六进制数字表示，而且加前缀“U+”。例如，字母 A 的码位是 U+0041，欧元符号的码位是 U+20AC，高音
谱号的码位是 U+1D11E。在 Unicode 6.3 中（这是 Python 3.4 使用的标准），约 10% 的有效码位有对应的字符。

* 字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换时使用的算法。在 UTF-8 编码中，A（U+0041）的码位编码成单个字节 \x41，而在 UTF-16LE 编码中编码成两个字节
\x41\x00。

> 把码位转换成字节序列的过程是编码；把字节序列转换成码位的过程是解码。

In [5]:
s = 'café'
print("len:", len(s))

b = s.encode('utf-8')
b
print("len of b:", len(b))
b.decode('utf-8')

len: 4
len of b: 5


'café'

> 如果想帮助自己记住 .decode() 和 .encode() 的区别，可以把字节序列想成晦涩难懂的机器磁芯转储，把 Unicode 字符串想成“人类可读”的文本。那么，把字节序列变成人类可读的文本字符串就是解码，而把字符串变成用于存储或传输的字节序列就是编
码。

虽然 Python 3 的 str 类型基本相当于 Python 2 的 unicode 类型，只不过是换了个新名称，但是 Python 3 的 bytes 类型却不是把 str 类型换个名称那么简单，而且还有关系紧密的 bytearray 类型。

## 4.2 字节概要

新的二进制序列类型在很多方面与 Python 2 的 str 类型不同。首先要知道，Python 内置了两种基本的二进制序列类型：Python 3 引入的不可变 bytes 类型和 Python 2.6 添加的可变 bytearray 类型。

bytes 或 bytearray 对象的各个元素是介于 0~255（含）之间的整数，而不像 Python 2 的 str 对象那样是单个的字符。然而，二进制序列的切片始终是同一类型的二进制序列，包括长度为 1 的切片

In [6]:
cafe = bytes('café', encoding='utf_8')

In [16]:
cafe[0]

99

In [17]:
cafe[:1]

b'c'

In [19]:
cafe_array = bytearray(cafe)
cafe_array

bytearray(b'caf\xc3\xa9')

In [20]:
cafe_array[-1:]

bytearray(b'\xa9')

In [24]:
cafe_array[-1]

169

虽然二进制序列其实是整数序列，但是它们的字面量表示法表明其中有ASCII 文本。因此，各个字节的值可能会使用下列三种不同的方式显示:

* 可打印的 ASCII 范围内的字节（从空格到 ~），使用 ASCII 字符本身。
* 制表符、换行符、回车符和 \ 对应的字节，使用转义序列\t、\n、\r 和 \\。
* 其他字节的值，使用十六进制转义序列（例如，\x00 是空字节）。

因此，我们看到的是 b'caf\xc3\xa9'：前 3 个字节b'caf' 在可打印的 ASCII 范围内，后两个字节则不然。

## 4.3 基本的编解码器

Python 自带了超过 100 种编解码器（codec, encoder/decoder），用于在文本和字节之间相互转换。

每个编解码器都有一个名称，如 'utf_8'，而且经常有几个别名，如 'utf8'、'utf-8' 和 'U8'。这些名称可以传给 open()、str.encode()、bytes.decode() 等函数的 encoding 参数。

In [28]:
for codec in ['utf-8', 'utf-16', 'gbk']:
    # print(codec, 'El Niño'.encode(codec), sep='\t')
    print(codec, '晴天'.encode(codec), sep='\t')

utf-8	b'\xe6\x99\xb4\xe5\xa4\xa9'
utf-16	b'\xff\xfetf)Y'
gbk	b'\xc7\xe7\xcc\xec'


## 4.4 了解编解码问题

虽然有个一般性的 UnicodeError 异常，但是报告错误时几乎都会指明具体的异常：UnicodeEncodeError（把字符串转换成二进制序列时）或 UnicodeDecodeError（把二进制序列转换成字符串时）。如果源码的编码与预期不符，加载 Python 模块时还可能抛出 SyntaxError。

> 出现与 Unicode 有关的错误时，首先要明确异常的类型。导致编码问题的是 UnicodeEncodeError、UnicodeDecodeError，还是如 SyntaxError 的其他错误？解决问题之前必须清楚这一点。

### 4.4.1 处理UnicodeEncodeError

多数非 UTF 编解码器只能处理 Unicode 字符的一小部分子集。把文本转换成字节序列时，如果目标编码中没有定义某个字符，那就会抛出UnicodeEncodeError 异常，除非把 errors 参数传给编码方法或函数，对错误进行特殊处理。

In [30]:
city = 'São Paulo'
city.encode('utf-8')

b'S\xc3\xa3o Paulo'

In [31]:
city.encode('utf-16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [32]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [33]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [35]:
city.encode('cp437', errors='ignore')

b'So Paulo'

In [39]:
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#227;o Paulo'

In [38]:
city.encode('gbk', errors='replace')

b'S&#227;o Paulo'

### 4.4.2 处理UnicodeDecodeError

不是每一个字节都包含有效的 ASCII 字符，也不是每一个字符序列都是有效的 UTF-8 或 UTF-16。因此，把二进制序列转换成文本时，如果假设是这两个编码中的一个，遇到无法转换的字节序列时会抛出UnicodeDecodeError。

In [41]:
octets = b'Montr\xe9al'

In [42]:
octets.decode('cp1252')

'Montréal'

In [43]:
octets.decode('iso8859_7')

'Montrιal'

In [44]:
octets.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [45]:
octets.decode('utf-8', errors='ignore')

'Montral'

In [46]:
octets.decode('utf-8', errors='replace')

'Montr�al'

### 4.4.3 使用预期之外的编码加载模块时抛出的SyntaxError

Python 3 默认使用 UTF-8 编码源码，Python 2（从 2.5 开始）则默认使用ASCII。如果加载的 .py 模块中包含 UTF-8 之外的数据，而且没有声明编码，会得到类似下面的消息：

GNU/Linux 和 OS X 系统大都使用 UTF-8，因此打开在 Windows 系统中使用 cp1252 编码的 .py 文件时可能发生这种情况。注意，这个错误在Windows 版 Python 中也可能会发生，因为 Python 3 为所有平台设置的默认编码都是 UTF-8。

In [47]:
print('Olá, Mundo!')

Olá, Mundo!



### 4.4.4 如何找出字节序列的编码

二进制序列编码文本通常不会明确指明自己的编码，但是 UTF 格式可以在文本内容的开头添加一个字节序标记。

## 4.5 处理文本文件

处理文本的最佳实践是“Unicode 三明治”（如图 4-2 所示）。 意思是，要尽早把输入（例如读取文件时）的字节序列解码成字符串。这种三明治中的“肉片”是程序的业务逻辑，在这里只能处理字符串对象。在其他处理过程中，一定不能编码或解码。对输出来说，则要尽量晚地把字符
串编码成字节序列。多数 Web 框架都是这样做的，使用框架时很少接触字节序列。例如，在 Django 中，视图应该输出 Unicode 字符串；Django 会负责把响应编码成字节序列，而且默认使用 UTF-8 编码。

> 在 Python 3 中能轻松地采纳 Unicode 三明治的建议，因为内置的 open函数会在读取文件时做必要的解码，以文本模式写入文件时还会做必要的编码，所以调用 my_file.read() 方法得到的以及传给my_file.write(text) 方法的都是字符串对象。

In [50]:
open('cafe.txt', 'w', encoding='utf-8').write('café')

4

In [53]:
open('cafe.txt', encoding='utf-8').read()

'café'

需要在多台设备中或多种场合下运行的代码，一定不能依赖默认编码。打开文件时始终应该明确传入 encoding= 参数，因为不同的设备使用的默认编码可能不同，有时隔一天也会发生变化。

In [54]:
fp = open('chapter_cate.txt', 'w', encoding='utf-8')

In [55]:
fp

<_io.TextIOWrapper name='chapter_cate.txt' mode='w' encoding='utf-8'>

In [56]:
fp.write('café')

4

In [57]:
fp.close()

In [59]:
import os

os.stat('chapter_cate.txt').st_size

5

In [60]:
fp2 = open('chapter_cate.txt')

In [61]:
fp2

<_io.TextIOWrapper name='chapter_cate.txt' mode='r' encoding='cp936'>

In [62]:
fp2.encoding

'cp936'

In [63]:
fp2.read()

'caf茅'

In [64]:
fp3 = open('chapter_cate.txt', encoding='utf-8')

In [65]:
fp3.encoding

'utf-8'

In [67]:
fp3.read()

'café'

In [68]:
fp4 = open('chapter_cate.txt', 'rb')

In [70]:
fp4

<_io.BufferedReader name='chapter_cate.txt'>

In [69]:
fp4.read()

b'caf\xc3\xa9'